In [1]:
import numpy as np
import pandas as pd

In [123]:
filename = "data_electre/matrix.csv"
data = pd.read_csv(filename, index_col = 'A')
p = pd.read_csv("data_electre/p_values.csv").values[0]
q = pd.read_csv("data_electre/q_values.csv").values[0]
v = pd.read_csv("data_electre/v_values.csv").values[0]
b_h = pd.read_csv("data_electre/b_values.csv", index_col = 'C').to_numpy()
data = data.to_numpy()

In [6]:
m, n = data.shape
b_n = b_h.shape[0]
b_n

2

In [4]:
b_h

[[100.   9.]
 [120.   7.]
 [150.   2.]
 [110.   5.]]


In [132]:
cj_ab = np.zeros([n,m,b_n])
cj_ba = np.zeros([n,m,b_n])

def concordance_index(a, b, q_j, p_j):
    diff = b-a
    if diff <= q_j:
        return 1
    elif diff >= p_j:
        return 0
    else:
        return (p_j - diff) / (p_j - q_j)

for j in range(n):
    q_j, p_j = q[j], p[j]
    for i_a in range(m):
        a = data[i_a,j]
        for i_b in range(b_n):
            b = b_h[j,i_b]
            cj_ab[j,i_a,i_b] = concordance_index(a,b,q_j,p_j)
            cj_ba[j,i_a,i_b] = concordance_index(b,a,q_j,p_j)

cj_ba, cj_ab

(array([[[0.  , 0.  ],
         [0.  , 0.  ],
         [0.  , 0.  ],
         [0.  , 0.  ]],
 
        [[0.  , 0.  ],
         [0.  , 0.  ],
         [0.75, 0.75],
         [0.  , 0.  ]]]),
 array([[[1., 1.],
         [1., 1.],
         [1., 1.],
         [1., 1.]],
 
        [[1., 1.],
         [1., 1.],
         [1., 1.],
         [1., 1.]]]))

In [125]:
dj_ab = np.zeros([n,m,b_n])
dj_ba = np.zeros([n,m,b_n])

def discordance_index(a, b, p_j, v_j):
    diff = b-a
    if diff <= p_j:
        return 0
    elif diff >= v_j:
        return 1
    else:
        return ((diff-p_j)/(v_j-p_j))

for j in range(n):
    p_j, v_j = p[j], v[j]
    for i_a in range(m):
        a = data[i_a,j]
        for i_b in range(b_n):
            b = b_h[j,i_b]
            dj_ab[j,i_a,i_b] = discordance_index(a,b,p_j,v_j)
            dj_ba[j,i_a,i_b] = discordance_index(b,a,p_j,v_j)

dj_ab, dj_ba

(array([[[0., 0.],
         [0., 0.],
         [0., 0.],
         [0., 0.]],
 
        [[0., 0.],
         [0., 0.],
         [0., 0.],
         [0., 0.]]]),
 array([[[1., 1.],
         [1., 1.],
         [1., 1.],
         [1., 1.]],
 
        [[1., 1.],
         [1., 1.],
         [0., 0.],
         [0., 0.]]]))

In [126]:
from smaa.sample import sample_weights

weights = np.array(sample_weights(n))
weights

array([0.27345653, 0.72654347])

In [133]:
C_ab = np.sum(cj_ab * weights[:,None,None], axis=0) / np.sum(weights)
C_ba = np.sum(cj_ba * weights[:,None,None], axis=0) / np.sum(weights)

# print(C_ab)
# a = np.array([[[1,2],[3,4]],[[5,6],[7,8]]])
# np.all((dj_ab[:,0,0] == 0))

C_ab

array([[1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.]])

In [140]:
def credibility_index(dj, C):
    if np.all((dj == 0.)):
        return C
    if np.any((dj == 1.)):
        return 0.
    diff = 1. - C
    result = C
    for d in dj:
        if dj > C:
            result *= (1-d)/diff
    return result

def outranks(dj, C):
    return credibility_index(dj, C) >= l

def preferred(dj, C):
    return outranks(dj[0], C[0]) and not outranks(dj[1], C[1])


def optimistic_procedure():
    mapper = np.zeros([m])
    for i in range(m):
        for j in range(b_n):
            if j == b_n-1:
                mapper[i] = b_n
            C = C_ba[i,j], C_ab[i,j]
            dj = dj_ba[:,i,j], dj_ab[:,i,j]
            if preferred(dj, C):
                mapper[i] = j
                break
    return mapper


def pessimistic_procedure():
    mapper = np.zeros([m])
    for i in range(m):
        for j in range(b_n-1,-1,-1):
            C = C_ab[i,j]
            dj = dj_ab[:,i,j]
            if outranks(dj, C):
                mapper[i] = j + 1
                break
    return mapper

l = 0.65

pessimistic_procedure()


array([2., 2., 2., 2.])

In [66]:
for i in range(m-2,-1,-1):
    print(i)

2
1
0
